In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import silhouette_score

In [2]:
outputbasefolder = '../../../Algorithms/Unsupervised_Segmentation/Approaches/With_Scribble/Outputs/Human_DLPFC/Report'

In [64]:
co_ordinates = pd.read_csv('Coordinates/coordinates.csv',index_col=0,header=0)
manual_annotation = pd.read_csv('manual_annotations.csv',index_col=0,header=0)
pc_s = pd.read_csv('Principal_Components/CSV/15_pcs.csv',index_col=0,header=0)
mclust_init = pd.read_csv('../../../Algorithms/Unsupervised_Segmentation/Approaches/With_Scribble/Outputs/Human_DLPFC/151673/many_models_1_0_0_itr_5/Outputs/Cluster_Labels_15final.csv',index_col=0,header=0)
combined_df = pd.concat([co_ordinates,manual_annotation,mclust_init,pc_s],axis=1)
# combined_df['label'].fillna(-1,inplace=True)
# combined_df = combined_df.dropna(subset=['label'])
combined_df['label'] = combined_df['manual_annotation']
combined_df.dropna(inplace=True)
# combined_df['label'] = combined_df['label'].astype(int)
# combined_df.drop(combined_df[combined_df['label']==40].index,inplace=True)


In [65]:
label_map = {
    0: 'WM',
    6: 'L1',
    5: 'L2',
    4: 'L3',
    3: 'L4',
    2: 'L5',
    1: 'L6',
    7: 'L7',
    -1: 'undetected',
}
# combined_df['label'] = combined_df['label'].map(label_map)

In [66]:
layers_dfs = []
inverse_layers_dfs = []
for i in sorted(combined_df['label'].unique()):
    layers_dfs.append(combined_df[combined_df['label']==i])
    inverse_layers_dfs.append(combined_df[combined_df['label']!=i])


In [67]:
pairwise_cosine_similarity = {}
for i in range(len(layers_dfs)):
    for j in range(len(layers_dfs)):
        cx = layers_dfs[i]['label'].unique()[0]
        cy = layers_dfs[j]['label'].unique()[0]
        if (cx,cy) not in pairwise_cosine_similarity:

            princs_i = layers_dfs[i][pc_s.columns.values].values
            princs_j = layers_dfs[j][pc_s.columns.values].values
            similarity = cosine_similarity(princs_i,princs_j)
            similarity = pd.DataFrame(similarity)
            # similarity.to_csv(outputbasefolder+'/_'+'layer_'+str(i)+'_layer_'+str(j)+'_cosine_similarity.csv')
            pairwise_cosine_similarity[(cx,cy)] = similarity

In [68]:
one_vs_all_cosine_similarity = {}
for i in range(len(layers_dfs)):
    cx = layers_dfs[i]['label'].unique()[0]

    princs_i = layers_dfs[i][pc_s.columns.values].values
    princs_j = inverse_layers_dfs[i][pc_s.columns.values].values
    similarity = cosine_similarity(princs_i,princs_j)
    similarity = pd.DataFrame(similarity)
    # similarity.to_csv(outputbasefolder+'/'+'one_vs_all_cosine_similarity_layer_'+str(i)+'.csv')
    one_vs_all_cosine_similarity[cx] = similarity

In [69]:
cos_sim_df = pd.DataFrame(columns=['layer_no','layer_1_count','layer_1_avg','others_count','others_avg','distance'])
for i in range(len(layers_dfs)):
    cx = layers_dfs[i]['label'].unique()[0]

    layer_i_cosine_similarity = pairwise_cosine_similarity[(cx,cx)].values.mean()
    layer_other_cosine_similarity = one_vs_all_cosine_similarity[cx].values.mean()

    # print('Layer '+str(i+1)+' cosine similarity: '+str(layer_i_cosine_similarity))
    # print('Layer '+str(i+1)+' one vs all cosine similarity: '+str(layer_other_cosine_similarity))
    avrage = layer_i_cosine_similarity-layer_other_cosine_similarity
    # print("distance: "+str(avrage))

    cos_sim_df.loc[i] = [cx,len(layers_dfs[i]),layer_i_cosine_similarity,len(inverse_layers_dfs[i]),layer_other_cosine_similarity,avrage]
# cos_sim_df.to_csv(outputbasefolder+'/cosine_similarity_onv_vs_all_33538_genes.csv')
cos_sim_df

,layer_no,layer_1_count,layer_1_avg,others_count,others_avg,distance
0,0.0,690.0,0.757471,2915.0,-0.433597,1.191069
1,3.0,978.0,0.484126,2627.0,0.026112,0.458014
2,4.0,924.0,0.519617,2681.0,0.031865,0.487752
3,5.0,619.0,0.241090,2986.0,0.043708,0.197382
4,7.0,394.0,0.151391,3211.0,0.025181,0.126209


In [70]:
cos_sim_df = pd.DataFrame(columns=['layer_no_1', 'layer_1_count', 'layer_1_avg', 'layer_no_2',
                          'layer_2_count', 'layer_2_avg', 'inter_layer_average_cos_sim', 'distance'])
for (i, j) in pairwise_cosine_similarity:
    layer_i_cosine_similarity = pairwise_cosine_similarity[(i,i)].values.mean()
        
    layer_other_cosine_similarity = pairwise_cosine_similarity[(j,j)].values.mean()

    layer_i_j_cosine_similarity = pairwise_cosine_similarity[(i, j)].values.mean()
    avrage = layer_i_cosine_similarity-layer_other_cosine_similarity
    # print("distance: "+str(avrage))

    # cos_sim_df.loc[i] = [i+1,len(layers_dfs[i]),layer_i_cosine_similarity,j+1,len(layers_dfs[j]),layer_other_cosine_similarity,layer_i_j_cosine_similarity,avrage]
    new_row = pd.DataFrame({'layer_no_1': i, 'layer_1_count': len(pairwise_cosine_similarity[(i,i)]), 'layer_1_avg': layer_i_cosine_similarity, 'layer_no_2': j, 'layer_2_count': len(
        pairwise_cosine_similarity[(j,j)]), 'layer_2_avg': layer_other_cosine_similarity, 'inter_layer_average_cos_sim': layer_i_j_cosine_similarity, 'distance': avrage},index=[f'{i}_{j}'])
    cos_sim_df = pd.concat([cos_sim_df, new_row], axis=0,ignore_index=True)
# cos_sim_df.to_csv(outputbasefolder+'/cosine_similarity_pairwise_33538_genes.csv')
cos_sim_df


,layer_no_1,layer_1_count,layer_1_avg,layer_no_2,layer_2_count,layer_2_avg,inter_layer_average_cos_sim,distance
0,0,690,0.757471,0,690,0.757471,0.757471,0.0
1,0,690,0.757471,3,978,0.484126,-0.54681,0.273346
2,0,690,0.757471,4,924,0.519617,-0.531466,0.237854
3,0,690,0.757471,5,619,0.24109,-0.282419,0.516381
4,0,690,0.757471,7,394,0.151391,-0.160568,0.606081
5,3,978,0.484126,0,690,0.757471,-0.54681,-0.273346
6,3,978,0.484126,3,978,0.484126,0.484126,0.0
7,3,978,0.484126,4,924,0.519617,0.341648,-0.035492
8,3,978,0.484126,5,619,0.24109,0.161599,0.243035
9,3,978,0.484126,7,394,0.151391,0.076605,0.332735
